# Neurorobotics with PyNN and PyBullet

In this Notebook we will build and execute a neuro-robotics experiment. Für the neural simulation we will use [PyNN](https://neuralensemble.org/PyNN/) and [Nest](https://www.nest-initiative.org/?page=Software). For the simulation of the physic we use [PyBullet](https://github.com/bulletphysics/bullet3/tree/master/examples/pybullet). 

This experiment is a re-creation of the [braitenberg-husky-experiment](https://bitbucket.org/hbpneurorobotics/experiments/src/development/braitenberg_husky/) in the neurorobotics plattform of the human brain project. 

We start by importing all modules we will use in this notebook. Make sure to install only python**3**-versions of all dependencies. 

You can not just install these via pip:

- _pybullet_ needs to be installed from source to get numpy-support. Otherwise it will be slower, and getCameraImage doesn't return a numpy-array. 
- _nest_ can only be installed from source. We must use version 2.16, because the current master is not yet compatible with _pynn_. Also _nest_ is compiled with _libnreuosim_, which needs a workaround until [the PR](https://github.com/nest/nest-simulator/pull/1235) is merged. I'm not 100% certain if _libneurosim_ is even required for this project, but _nest_ gives a warning if it's missing, so we will install it. 
- there is a warning "UserWarning: Unable to install NEST extensions. Certain models may not be available", which doesn't seem to affect this project. Please ignore it. 

In [1]:
import time
import numpy as np

import pyNN.nest as brain_sim
import csv

from matplotlib import pyplot as plt
from matplotlib import animation
from pyNN.utility.plotting import Figure, Panel
from quantities import mV

/opt/conda/lib/python3.7/site-packages/pyNN/nest/__init__.py:55: UserWarning: Unable to install NEST extensions. Certain models may not be available.
Further details: DynamicModuleManagementError in Install: Module 'pynn_extensions' could not be opened.
The dynamic loader returned the following error: 'file not found'.

Please check LD_LIBRARY_PATH (OSX: DYLD_LIBRARY_PATH)!
  warnings.warn("Unable to install NEST extensions. Certain models may not be available.\nFurther details: {}".format(err))


### brain sim mit pynn

first we define the population in pynn. This code is copied from [braitenberg.py on hbpneurorobotics](https://bitbucket.org/hbpneurorobotics/models/src/development/brains/braitenberg.py)

In [2]:
class brain(object):
    
    def __init__(self): 
        sim = brain_sim
        SENSORPARAMS = {'v_rest': -60.5,
                        'cm': 0.025,
                        'tau_m': 10.,
                        'tau_refrac': 10.0,
                        'tau_syn_E': 2.5,
                        'tau_syn_I': 2.5,
                        'e_rev_E': 0.0,
                        'e_rev_I': -75.0,
                        'v_thresh': -60.0,
                        'v_reset': -60.5}

        GO_ON_PARAMS = {'v_rest': -60.5,
                        'cm': 0.025,
                        'tau_m': 10.0,
                        'e_rev_E': 0.0,
                        'e_rev_I': -75.0,
                        'v_reset': -61.6,
                        'v_thresh': -60.51,
                        'tau_refrac': 10.0,
                        'tau_syn_E': 2.5,
                        'tau_syn_I': 2.5}

        self.population = sim.Population(8, sim.IF_cond_alpha())
        self.population[0:5].set(**SENSORPARAMS)
        self.population[5:6].set(**GO_ON_PARAMS)
        self.population[6:8].set(**SENSORPARAMS)

        syn_params = {'U': 1.0, 'tau_rec': 1.0, 'tau_facil': 1.0}

        # Synaptic weights originale
        WEIGHT_RED_TO_ACTOR = 1.5e-4
        WEIGHT_RED_TO_GO_ON = 1.2e-3  # or -1.2e-3?
        WEIGHT_GREEN_BLUE_TO_ACTOR = 1.05e-4
        WEIGHT_GO_ON_TO_RIGHT_ACTOR = 1.4e-4
        DELAY = 1

        # Connect neurons
        CIRCUIT = self.population

        SYN = sim.TsodyksMarkramSynapse(weight=abs(WEIGHT_RED_TO_ACTOR),
                                        delay=DELAY, **syn_params)
        self._p1 = sim.Projection(presynaptic_population=CIRCUIT[2:3], 
                       postsynaptic_population=CIRCUIT[7:8],
                       connector=sim.AllToAllConnector(),
                       synapse_type=SYN,
                       receptor_type='excitatory')
        self._p2 = sim.Projection(presynaptic_population=CIRCUIT[3:4],
                       postsynaptic_population=CIRCUIT[6:7],
                       connector=sim.AllToAllConnector(),
                       synapse_type=SYN,
                       receptor_type='excitatory')


        SYN = sim.TsodyksMarkramSynapse(weight=abs(WEIGHT_RED_TO_GO_ON),
                                        delay=DELAY, **syn_params)
        self._p3 = sim.Projection(presynaptic_population=CIRCUIT[0:2],
                       postsynaptic_population=CIRCUIT[4:5],
                       connector=sim.AllToAllConnector(),
                       synapse_type=SYN,
                       receptor_type='inhibitory')
        self._p4 = sim.Projection(presynaptic_population=CIRCUIT[0:2],
                       postsynaptic_population=CIRCUIT[5:6],
                       connector=sim.AllToAllConnector(),
                       synapse_type=SYN,
                       receptor_type='inhibitory')

        SYN = sim.TsodyksMarkramSynapse(weight=abs(WEIGHT_GREEN_BLUE_TO_ACTOR),
                                        delay=DELAY, **syn_params)
        self._p5 = sim.Projection(presynaptic_population=CIRCUIT[4:5],
                       postsynaptic_population=CIRCUIT[7:8],
                       connector=sim.AllToAllConnector(),
                       synapse_type=SYN,
                       receptor_type='excitatory')

        SYN = sim.TsodyksMarkramSynapse(weight=abs(WEIGHT_GO_ON_TO_RIGHT_ACTOR),
                                        delay=DELAY, **syn_params)
        self._p6 = sim.Projection(presynaptic_population=CIRCUIT[5:6],
                       postsynaptic_population=CIRCUIT[7:8],
                       connector=sim.AllToAllConnector(),
                       synapse_type=SYN,
                       receptor_type='excitatory')

    def get_pop(self):  
        return self.population   
    
    def get_weights(self):
        w1 = self._p1.get('weight', format='array')
        w2 = self._p2.get('weight', format='array')
        w3 = self._p3.get('weight', format='array')
        w4 = self._p4.get('weight', format='array')
        w5 = self._p5.get('weight', format='array')
        w6 = self._p6.get('weight', format='array')
        
        return w1, w2, w3, w4, w5, w6
    
    def set_weights(self, WEIGHT_RED_TO_ACTOR, WEIGHT_RED_TO_GO_ON, WEIGHT_GREEN_BLUE_TO_ACTOR, WEIGHT_GO_ON_TO_RIGHT_ACTOR):
        self._p1.set(weight=WEIGHT_RED_TO_ACTOR)
        self._p2.set(weight=WEIGHT_RED_TO_ACTOR)
        self._p3.set(weight=WEIGHT_RED_TO_GO_ON)
        self._p4.set(weight=WEIGHT_RED_TO_GO_ON)
        self._p5.set(weight=WEIGHT_GREEN_BLUE_TO_ACTOR)
        self._p6.set(weight=WEIGHT_GO_ON_TO_RIGHT_ACTOR)

        
        

next we set up the input for the population. For this we create two poisson spike generators and hook them up to the population according to original project. The values for the projections come from in the first 6 lines of [this file](https://bitbucket.org/hbpneurorobotics/experiments/src/development/braitenberg_husky/eye_sensor_transmit.py) and the parameters for the new neurons come are the [default values for PoissonSpikeGenerators](https://bitbucket.org/hbpneurorobotics/cle/src/development/hbp_nrp_cle/hbp_nrp_cle/brainsim/pynn/devices/__PyNNPoissonSpikeGenerator.py)

In [3]:
class SpikeGenerator(object):
    """the class handles the input for the brain"""
    
    def __init__(self, population, interval=20.0, count_red_left=1/3, count_red_right=1/3, count_non_red=1/3):
        params_source=  {'start': 0.0, 'duration': float("inf"), 'rate': 0.0}
        params_connec= {'weight':0.00015, 'receptor_type':'excitatory','delay':0.1}
        self.ssp_red_left_eye = brain_sim.create(brain_sim.SpikeSourcePoisson,  params_source)
        self.ssp_red_right_eye = brain_sim.create(brain_sim.SpikeSourcePoisson,  params_source)
        self.ssp_green_blue_eye = brain_sim.create(brain_sim.SpikeSourcePoisson,  params_source)
        brain_sim.connect(self.ssp_red_left_eye, population[slice(0, 3, 2)], **params_connec)
        brain_sim.connect(self.ssp_red_right_eye, population[slice(1, 4, 2)], **params_connec)
        brain_sim.connect(self.ssp_green_blue_eye, population[4], **params_connec)
        
    def set_rates_from_ratios(self, red_left_rate, red_right_rate, non_red_rate):
        try:
            self.ssp_red_left_eye.set(rate=red_left_rate)
            self.ssp_red_right_eye.set(rate=red_right_rate)
            self.ssp_green_blue_eye.set(rate=non_red_rate)
        except StopIteration:
            pass

the next class handles the output of the brain. It connects two leaky-integrator neurons tp the output-neurons to convert their spikes into voltages, which we will measure. 
        
        
The parameters for these Neurons come from [defaults](https://bitbucket.org/hbpneurorobotics/cle/src/aff5aa918fdb6a07cb1ede3112f662ed679fccc4/hbp_nrp_cle/hbp_nrp_cle/brainsim/pynn/devices/__PyNNLeakyIntegratorTypes.py#lines-44), again. The connection to the population come from [line 10 and 11 of this file from the original experiment](https://bitbucket.org/hbpneurorobotics/experiments/src/d6e01275e51e0db8fc341bfe5873cb805d4c5c44/braitenberg_husky/braitenberg_husky_linear_twist.py#lines-10)


In [4]:
class OutputGenerator(object):
    def __init__(self, population):
        LI_create = {'v_thresh': float('inf'),'cm': 1.0,'tau_m': 10.0,'tau_syn_E': 2.,
            'tau_syn_I': 2.,'v_rest': 0.0,'v_reset': 0.0,'tau_refrac': 0.1,}
        LI_connect = {'weight':0.00015, 'receptor_type':'excitatory','delay':0.1}
        self.left = brain_sim.create(brain_sim.IF_curr_alpha, LI_create)
        self.right = brain_sim.create(brain_sim.IF_curr_alpha, LI_create)
        brain_sim.connect(population[6], self.left, **LI_connect)
        brain_sim.connect(population[7], self.right,  **LI_connect)
        brain_sim.initialize(self.left, v=self.left.get('v_rest'))
        brain_sim.initialize(self.right, v=self.right.get('v_rest'))
        self.left.record('v')
        self.right.record('v')
    
    def get_current_voltage(self):
        return self._get_current_voltage(self.left), self._get_current_voltage(self.right)
    
    def _get_current_voltage(self, cell):
        return cell.get_data('v', clear=True).segments[0].filter(name="v")[0][-1,0].item()

# Import csv to list

In [5]:
with open('output1.csv', 'r') as f:
    reader = csv.reader(f)
    logging_information_list = list(reader)

# Convert to numpy array 
logging_information = np.asarray(logging_information_list, dtype=np.double)

# Run experiment

now that we have defined everything we need, lets boot up experiment.

We will work with synchronized timesteps in both the brain simulation and the physics simulation. The default in bullet is [1ms](https://github.com/bulletphysics/bullet3/blob/d220101c5aad92cf7013710b0ed011395cabbb23/examples/pybullet/pybullet.c#L3001). We will be much slower than the defaults. On one hand we only need to update our inputs only a couple of times per seconds. On the other hand the rendering of the robot-images and the updating of the motor-input takes up a significant amount of processing. 

if the timesteps are too short, the simulation is very slow. If the timesteps are too long, the robot won't notice the red-area until it's too late. 

In [6]:
timestep_ms = 7

#### initialize the brain simulation 

Note: the _timestep_ variable in brain_sim.setup() referes to the internal timesteps, and not to the external kind 

In [7]:
brain_sim.setup(timestep=0.1,min_delay=0.1,max_delay=4.0)
b = brain()
population=b.get_pop()
brain_sim.initialize(population, v=population.get('v_rest'))
spike_generator = SpikeGenerator(population)
output_generator = OutputGenerator(population)

#### run experiment

and now lets to a single step of the simulation. We wrap it in a function, so we can repeat the step at later points. 

Each step does the following: 

1. do a single step in the physics simulation
2. extract an image from the robot's camera
3. use that image to set the input for the brain simulation
4. do a single step in the brain simulation
5. extract the output from the brain
6. use that output to set motor-controls for the robot
7. return extracted image and voltages

the returned values may later be used to visualize what's happening. 

In [8]:
def step(*ratio):
    
    # Set the rates from the image
    spike_generator.set_rates_from_ratios(*ratio)
    
    # Run one step in the brain simulation
    brain_sim.run(timestep_ms)
        
    # Get the current voltage of the output neurons
    voltage_left, voltage_right = output_generator.get_current_voltage() #10ms    
    
    return voltage_left, voltage_right

In [9]:
output = []
voltage_left_error = 0
voltage_right_error = 0

for row in logging_information:
    
    # Get logging information from csv file
    age, voltage_left1, voltage_right1, red_left_rate, red_right_rate, non_red_rate = row
    
    # Simulate
    voltage_left2, voltage_right2 = step(red_left_rate, red_right_rate, non_red_rate)
    
    # Calculate errors
    voltage_left_diff = abs(voltage_left2 - voltage_left1)
    voltage_left_error += voltage_left_diff
    voltage_right_diff = abs(voltage_right2 - voltage_right1)
    voltage_right_error += voltage_right_diff
    
    # Write into output array
    output.append([voltage_left2, voltage_right2])
    
print(voltage_left_error)
print(voltage_right_error)

0.0
0.0


# Save Array

In [10]:
with open("output2.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(output)

In [11]:
print(b.get_weights())

(array([[0.00015]]), array([[0.00015]]), array([[0.0012],
       [0.0012]]), array([[0.0012],
       [0.0012]]), array([[0.000105]]), array([[0.00014]]))


In [12]:
b.set_weights(WEIGHT_RED_TO_ACTOR = 1.5e-2, WEIGHT_RED_TO_GO_ON = 1.2e-2, WEIGHT_GREEN_BLUE_TO_ACTOR = 1.05e-3, WEIGHT_GO_ON_TO_RIGHT_ACTOR = 1.4e-4)

In [13]:
print(b.get_weights())

(array([[0.015]]), array([[0.015]]), array([[0.012],
       [0.012]]), array([[0.012],
       [0.012]]), array([[0.00105]]), array([[0.00014]]))
